# NIFTY50 Stock Summary (Table 1)

### 🎯 Objective
In this notebook, we will:
1. Scrape **NIFTY50 stock summary** from Yahoo Finance.
2. Save the **raw dataset** in `/data/raw/`.
3. Clean and standardize the dataset:
   - Normalize column names
   - Convert numeric columns
   - Handle missing values
4. Save the **cleaned dataset** in `/data/cleaned/`.


----
## Starting with Phase 1
----

## Step 1: Setup Directories and Import Libraries

We begin by importing Python libraries and creating dedicated directories to keep our raw data and cleaned data organized.

In [1]:
# Step 1: Import Libraries
import pandas as pd
import numpy as np
import os

# Create directories
os.makedirs("../data/raw", exist_ok=True)
os.makedirs("../data/cleaned", exist_ok=True)


## Step 2: Retrieve Raw Data from NSE API

The NSE India API provides structured JSON data of NIFTY 50 constituents.
We send a request with a browser-like User-Agent header to prevent the server from blocking our request.

URL: https://www.nseindia.com/market-data/live-equity-market?symbol=NIFTY%2050


In [2]:
import requests
url = "https://www.nseindia.com/api/equity-stockIndices?index=NIFTY%2050"
headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/120.0.0.0 Safari/537.36"
    )
}

response = requests.get(url, headers=headers)
response.raise_for_status()  # Ensure the request was successful
data_json = response.json()


HTTPError: 401 Client Error: Unauthorized for url: https://www.nseindia.com/api/equity-stockIndices?index=NIFTY%2050

## Step 3: Save Raw JSON for Reference

It’s good practice to store the raw unmodified data for traceability.
This ensures we always have the original source if something goes wrong in cleaning.

In [3]:
with open("../data/raw/stock_summary_raw.json", "w") as f:
    f.write(response.text)


## Step 4: Parse JSON into a DataFrame

The raw JSON contains a key "data" where all stock information is stored.
We will extract it and convert it into a structured pandas DataFrame.

In [7]:
records = data_json.get("data", [])
df_raw = pd.DataFrame(records)

print("Raw columns:", df_raw.columns)
df_raw.head()


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

## Step 5: Save Raw CSV Snapshot

Along with JSON, we save the raw CSV to make it easier for analysts who prefer working directly with tabular data

In [ ]:
df_raw.to_csv("../data/raw/stock_summary_raw.csv", index=False)


## Step 6: Clean and Standardize Column Names

Raw column names may contain spaces, mixed cases, and special characters.
We will normalize column names to lowercase, snake_case, and align them with our project schema.

In [1]:
df = df_raw.copy()

# Normalize column names
df.columns = (
    df.columns.str.strip()
    .str.lower()
    .str.replace(" ", "_")
    .str.replace(r"[^a-z0-9_]", "", regex=True)
)

# Rename selected fields to standard names
rename_map = {
    "symbol": "symbol",
    "lastprice": "last_price",
    "change": "change",
    "pchange": "perc_change",
    "open": "open",
    "totaltradedvolume": "volume",
    "totaltradedvalue": "value",
    
    # Additional mappings
    "dayhigh": "high",
    "daylow": "low",
    "previousclose": "prev_close",
    "ffmc": "ffmc",
    "yearhigh": "year_high",
    "yearlow": "year_low",
    "stockindcloseprice": "index_close_price",
    "lastupdatetime": "last_update",
    "nearwkh": "near_week_high",
    "nearwkl": "near_week_low",
    "perchange365d": "perc_change_365d",
    "date365dago": "date_365d_ago",
    "chart365dpath": "chart_365d_path",
    "date30dago": "date_30d_ago",
    "perchange30d": "perc_change_30d",
    "chart30dpath": "chart_30d_path",
    "charttodaypath": "chart_today_path",
    "series": "series",
    "meta": "meta"

}
df = df.rename(columns=rename_map)

df.head()


NameError: name 'df_raw' is not defined

## Step 7: Convert Numerical Columns

By default, some fields may be stored as strings.
We will explicitly convert them into numeric values for proper analysis.

In [ ]:
numeric_cols = ["last_price", "change", "perc_change", "open", "high", "low", "volume", "value"]

for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")


## Step 8: Handle Missing Values

To ensure data quality, we will handle missing values by:

Dropping rows where the stock symbol is missing.

Filling missing numerical fields with the median value of that column.

In [ ]:
# Drop invalid rows
df = df.dropna(subset=["symbol"])

# Fill missing numeric fields
for col in numeric_cols:
    if col in df.columns:
        df[col] = df[col].fillna(df[col].median())


### Further cleaning and standardising 

* Check the column headers to determine if further standardisation is required.
* Check for more missing values and replace them with a default  or a  suitable value.
* Check the datatypes of each date & time-specific column. If it's a string, convert it to datetime format.

In [ ]:
#Check the column headers
print(df.columns)


In [ ]:
#print(df.loc[8, 'perc_change_365d']) => Null
df.loc[8, 'perc_change_365d'] = 0
print(df.loc[8, 'perc_change_365d'])


In [ ]:
print(df.loc[8, 'date_365d_ago'])
df.loc[8, 'date_365d_ago'] = '05-Sep-2024'
print(df.loc[8, 'date_365d_ago'])

In [ ]:
df.loc[8, 'chart_365d_path'] = 'Missing.svg'
print(df.loc[8, 'chart_365d_path'])

#### Check for the datatype of date or time-specific columns 

In [ ]:
print(type('last_update'))

In [ ]:
print(type('date_30d_ago'))

#### Before conversion make sure all missing values are dealt with 

In [ ]:
# Get the first non-null value from last_update
last_update_value = df["last_update"].dropna().iloc[0]

# Fill all missing values with this timestamp
df["last_update"] = df["last_update"].fillna(last_update_value)

# Quick check
df["last_update"].unique()


In [ ]:
print(df.loc[8, 'last_update'])

#### After dealing with the missing values now convert their data types

In [ ]:
# Define date columns
date_cols = ["last_update", "date_365d_ago", "date_30d_ago"]

# Convert string → datetime
for col in date_cols:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors="coerce", dayfirst=False)

# Verify conversions
df.dtypes[["last_update", "date_365d_ago", "date_30d_ago"]]


## Step 9: Save the Final Cleaned Dataset

Finally, we save the cleaned dataset which is ready for analysis or visualization.

In [ ]:
df.to_csv("../data/cleaned/indstock_summary_cleaned.csv", index=False)
print("✅ Cleaned dataset saved successfully!")


----
## Starting with Phase 2
----

## Step 10: Now Extract The Meta Column

We will save the `meta` column dataset into `../data/raw`, naming it `indstock_metadata.csv`.

In [3]:
import pandas as pd
import ast

# ✅ Step 1: Load your cleaned stock summary file
df = pd.read_csv("../data/cleaned/indstock_summary_cleaned.csv")

# ✅ Step 2: Drop rows where 'meta' is empty
df_meta = df.dropna(subset=["meta"]).copy()

# ✅ Step 3: Convert 'meta' string into Python dictionary
df_meta["meta_dict"] = df_meta["meta"].apply(lambda x: ast.literal_eval(x))

# ✅ Step 4: Expand the dictionary into separate columns
meta_expanded = pd.json_normalize(df_meta["meta_dict"])

# ✅ Step 5: Keep symbol + metadata together
meta_final = pd.concat([df_meta[["symbol"]].reset_index(drop=True),
                        meta_expanded.reset_index(drop=True)], axis=1)

# ✅ Step 6: Save as new CSV file
meta_final.to_csv("../data/raw/indstock_metadata.csv", index=False)

print("Metadata extracted and saved to 'indstock_metadata.csv'")
print(meta_final.head())


AttributeError: partially initialized module 'pandas' from 'C:\Users\paula\Documents\My Workspace\Projects\IndianStockAnalysis\.env_ind_stock\Lib\site-packages\pandas\__init__.py' has no attribute 'core' (most likely due to a circular import)

####  Check for the Column heads

In [4]:
meta_final.columns


NameError: name 'meta_final' is not defined

## Step 11: Standardisation of Column Header Name 

Rename column header into a uniform pattern.

In [5]:
import pandas as pd

# Load file
df_meta = pd.read_csv("../data/raw/indstock_metadata.csv")

# Define rename mapping
rename_map = {
    "companyName": "company_name",
    "industry": "industry",
    "activeSeries": "active_series",
    "debtSeries": "debt_series",
    "isFNOSec": "is_fno_sec",
    "isCASec": "is_ca_sec",
    "isSLBSec": "is_slb_sec",
    "isDebtSec": "is_debt_sec",
    "isSuspended": "is_suspended",
    "tempSuspendedSeries": "temp_suspended_series",
    "isETFSec": "is_etf_sec",
    "isDelisted": "is_delisted",
    "isin": "isin",
    "slb_isin": "slb_isin",
    "listingDate": "listing_date",
    "isMunicipalBond": "is_municipal_bond",
    "isHybridSymbol": "is_hybrid_symbol",
    "quotepreopenstatus.equityTime": "equity_time",
    "quotepreopenstatus.preOpenTime": "pre_open_time",
    "quotepreopenstatus.QuotePreOpenFlag": "quote_pre_open_flag",
}

# Apply rename
df_meta = df_meta.rename(columns=rename_map)

# Remove duplicate "symbol" column if needed
df_meta = df_meta.loc[:, ~df_meta.columns.duplicated()]

# Save cleaned file
df_meta.to_csv("../data/cleaned/indstock_metadata_cleaned.csv", index=False)

print("✅ Standardized metadata saved as 'indstock_metadata_cleaned.csv'")
print(df_meta.head())


AttributeError: partially initialized module 'pandas' from 'C:\Users\paula\Documents\My Workspace\Projects\IndianStockAnalysis\.env_ind_stock\Lib\site-packages\pandas\__init__.py' has no attribute 'core' (most likely due to a circular import)

##


***

## 📊 Conclusion

We have successfully:
* Extracted the complete stock summary of all NIFTY 50 constituents.
* Organized the workflow into raw and cleaned datasets.
* Ensured numerical consistency and handled missing values.

This cleaned dataset can now be directly used for:
* **Exploratory Data Analysis (EDA)**
* **Visualization Dashboards**
* **Integration into larger financial projects**

---